In [27]:
# To run this example you'll need to install matplotlib.
# The easiest way to install is by opening a terminal
# and running `pip install --user matplotlib`.
#
# NOTE:
#   If your terminal doesn't recognize pip you might need to install it.
#   Search google for the pip (python package index) page to download get-pip.py
#   and follow the instructions for installing pip. If that doesn't work feel free to use pencil and paper.

import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt

In [28]:
# make sure you move plotting.ipynb into the likelihood-example directory so you
# can easily access data/eventsummary_4years.txt

import numpy as np

def loadEvents(filename):
    ID,E,EerrMin,EerrMax,DEC,RA,Anger = np.loadtxt(filename,comments='#',usecols=(0,1,2,3,5,6,7), unpack =True)
    #for i in range(len(ID)):
    #    print ID[i], DEC[i]
    topo = np.loadtxt(filename,comments='#',usecols=[8], unpack =True, dtype = 'str')
    
    array = np.recarray(len(E), [('topology', topo.dtype),
                                 ('energy', float),
                                 ('energy_error', float),
                                 ('RA', float),
                                 ('DEC', float),
                                 ('ID', int),
                                 ('angular_error', float)])
    array.ID[:] = ID
    array.topology[:] = topo
    array.energy[:], array.energy_error[:] = E, np.array([EerrMin, EerrMax]).max(axis=0)
    array.RA[:], array.DEC[:], array.angular_error[:] = np.radians(RA), np.radians(DEC), np.radians(Anger)
    
    return array

HESEevents = loadEvents("data/eventsummary_4years.txt")

# convert neutrino event location to degrees for plotting later (it's easier to visualize in degrees)
HESEevents.RA  = np.degrees( HESEevents.RA )
HESEevents.DEC = np.degrees( HESEevents.DEC )
HESEevents.angular_error = np.degrees( HESEevents.angular_error )

In [15]:
# plot events near HAWC source 2HWC J0631+169 (Geminga)
src_ra  = 98.00 # source right ascension in degrees
src_dec = 17.00 # source declination in degrees

# make a scatter plot of the events
plt.scatter(x=HESEevents.RA, y=HESEevents.DEC,
            s=10*HESEevents.angular_error, # s is the marker size, I set it to scale with angular error
            marker='o',color='g')


# zoom to region centered on source
dx = 10
plt.xlim(src_ra - dx, src_ra + dx)

dy = dx # setting dy = dx just draws a square
plt.ylim(src_dec - dy, src_dec + dy)

# print list of nearby events
mask = ((HESEevents.RA  > src_ra  - dx) & (HESEevents.RA  < src_ra  + dx)) & \
       ((HESEevents.DEC > src_dec - dy) & (HESEevents.DEC < src_dec + dy)) 
print(HESEevents[mask])
    
# this will create a picture inside the likelihood-example folder
# called event_locations.png. Open it and take a look. 
plt.savefig("event_locations.png")
plt.clf() # clear figure for making more plots later

[('Track',  200.5,  16.4,  93.3,  14., 38,  1.2)]


In [29]:
# values of TS for different assumed numbers of signal events (ns)
ns = [1, 2, 3, 4, 5]
TS = [1.87, 3.78, 5.73, 7.72, 9.74]

plt.scatter(x=ns, y=TS, marker='o')
plt.xlabel("Number of signal events (ns)")
plt.ylabel("Test Statistic (TS)")

# some limits that work with these values of ns, TS
# you'll probably need to change these to look at other sources
plt.xlim(0,5)
plt.ylim(0,10)

plt.savefig("ts_versus_ns.png")
plt.clf() # clear figure for making more plots later